In [35]:
import os
import json
from torch.cuda import empty_cache
import gc
import nibabel as nib
import numpy as np
from tqdm.auto import tqdm

from utils.modelUtils import load_sam, load_sammed2d
from classes.SAMClass import SAMWrapper, SAMInferer
from classes.SAMMed2DClass import SAMMed2DInferer, SAMMed2DWrapper

import utils.promptUtils as prUt
import utils.analysisUtils as anUt
import utils.testingUtils as tUt
from utils.imageUtils import read_im_gt

In [5]:
dataset_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub'
organ_labels_dict = None

In [32]:
if organ_labels_dict is None:
        with open(os.path.join(dataset_path, 'dataset.json'), 'r') as f:
            metadata = json.load(f)
            organ_labels_dict = metadata['labels']

images_dir = os.path.join(dataset_path, 'imagesTr')
labels_dir = os.path.join(dataset_path, 'labelsTr')
image_paths = sorted(os.path.join(images_dir, f) for f in os.listdir(images_dir))
labels_paths = sorted(os.path.join(labels_dir, f) for f in os.listdir(labels_dir))


for img_path, gt_path in tqdm(zip(image_paths, labels_paths), desc = 'Inferring'):
    for organ, organ_label in organ_labels_dict.items():
        if organ_label == 0 or organ == 'background':
            continue
        
        # Load in image, prompt
        img, gt = read_im_gt(img_path, gt_path, organ_label)

        d = tUt.gt_fg_dims(gt)
        if any([dim < 128 for dim in d.values()]):
            print(gt_path, organ, d)
        

Inferring: 0it [00:00, ?it/s]

/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz aorta {'Depth': 139, 'Height': 130, 'Width': 80}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz gall_bladder {'Depth': 12, 'Height': 71, 'Width': 62}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz kidney_left {'Depth': 45, 'Height': 75, 'Width': 91}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz kidney_right {'Depth': 41, 'Height': 81, 'Width': 89}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz liver {'Depth': 55, 'Height': 174, 'Width': 302}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz pancreas {'Depth': 25, 'Height': 108, 'Width': 164}
/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_sub (copy)/labelsTr/BDMAP_00000001.nii.gz pos

In [43]:
with open('/home/t722s/Desktop/inferenceResults/exp5pointperslice.json', 'r') as f:
    res = json.load(f)
res_sam = res['sam']
res_nums_sam = {k:np.nanmean(list(v.values())) for k, v in res_sam.items()}


res_sammed2d = res['sammed2d']
res_nums_sammed2d = {k:np.nanmean(list(v.values())) for k, v in res_sammed2d.items()}
res_nums_sam, res_nums_sammed2d


({'aorta': 0.42622881237084764,
  'gall_bladder': 0.417955873812016,
  'kidney_left': 0.5911278614523336,
  'liver': 0.277672827158039},
 {'aorta': 0.8010184030301172,
  'gall_bladder': 0.407296836419257,
  'kidney_left': 0.864371405254184,
  'liver': 0.9457547764211236})